In [12]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import (List)

embedding = OpenAIEmbeddings()
splitter = RecursiveCharacterTextSplitter()

documents = []
loaders = [
    PyPDFLoader("../data/autogpt.pdf"),
    PyPDFLoader("../data/lora.pdf"),
]

for loader in loaders:
    documents.extend(loader.load())

chunks = splitter.split_documents(documents)


In [3]:
persist_directory = '../docs/chroma/02'
vectordb = Chroma(
    embedding_function=embedding,
    persist_directory=persist_directory
)


if (len(vectordb.get()['ids']) < 1):
    print('persist...')
    vectordb.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory=persist_directory
    )


print(vectordb)



In [11]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

refine_qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type='refine',
    return_source_documents=True,
)

map_qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type='map_reduce',
    return_source_documents=True,
)

query = 'Make a summary in 200 words of lora paper' 
result = qa_chain({"query": query})
print(result["result"])


query = 'Make a summary in 200 words of lora paper' 
result = refine_qa_chain({"query": query})
print(result["result"])

query = 'Make a summary in 200 words of lora paper' 
result = map_qa_chain({"query": query})
print(result["result"])



query = 'What  is the biggest country in the world' 
result = qa_chain({"query": query})
print(result["result"])




The LoRA paper introduces an efficient adaptation strategy for large language models that allows for quick task-switching without introducing inference latency or reducing input sequence length. By adapting only a subset of weight matrices in a neural network, LoRA reduces memory and storage usage significantly, making it more cost-effective to deploy customized models for different tasks. The paper also explores the impact of LoRA on downstream task performance for various language models like RoBERTa, DeBERTa, and GPT-2, showcasing its effectiveness in improving model quality while reducing training time and resource requirements. Thanks for asking!
The additional context provided includes hyperparameters for DeBERTa XXL on tasks in the GLUE benchmark, hyperparameters for GPT-2 LoRA on E2E, WebNLG, and DART datasets, and details on parameter tuning for LoRA in combination with preﬁx-based approaches on WikiSQL and MNLI tasks. The hyperparameters include optimizer, learning rate, weig